In [ ]:
import os
file_dir = r"..\llm-20-questions"
file_path = file_dir + r"\data.md"
overview_dir = file_dir + "_jp"
overview_path = overview_dir + r"\overview.md"
output_dir = file_dir + "_jp"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
output_path = output_dir + r"\data.md"

In [ ]:
from calendar import c
from urllib import response
import nbformat
import os
import time
from tqdm import tqdm

class ResourceExhausted(Exception):
    pass
from prompt_toolkit import prompt
os.environ["GEMINI_API_KEY"] = "YOUR_API_KEY"
import google.generativeai as genai
genai.configure(api_key=os.environ["GEMINI_API_KEY"])
import re

generation_config = {
    "temperature": 0,
    "top_p": 0.95,
    "top_k": 64,
    "max_output_tokens": 8192,
    "response_mime_type": "text/plain",
}
with open(overview_path, 'r', encoding='utf-8') as file:
    content = file.read()

system_instruction = """あなたはプロの翻訳家でかつデータサイエンティストです。ユーザーから送られてくるkaggleのコンペティションのデータ説明を翻訳するのがあなたの役割です。ただし、日本語に翻訳する際は単に直訳するのではなく、日本語として自然な文章になるよう心がけてください。また、以下にコンペティションの情報を提供するので必要であれば用いてください。

コンペティションの概要:
```markdown
""" + content + "\n```"


model = genai.GenerativeModel(
    model_name="gemini-1.5-pro",
    generation_config=generation_config,
    system_instruction=system_instruction
)

def split_text_by_tokens(file_path, target_tokens=4000):

    # ファイルの読み込み
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
    model = genai.GenerativeModel(
        model_name="gemini-1.5-pro",
        generation_config=generation_config,
    )
    chunks = []
    current_chunk = ""
    current_tokens = 0

    for line in content.split('\n'):
        # 空の行をスキップ
        if not line.strip():
            continue
        line_tokens = model.count_tokens(line).total_tokens
        
        if current_tokens + line_tokens > target_tokens and current_chunk:
            chunks.append(current_chunk.strip())
            current_chunk = ""
            current_tokens = 0

        current_chunk += line + '\n'
        current_tokens += line_tokens

    if current_chunk:
        chunks.append(current_chunk.strip())

    #全てのチャンクのトークン数を確認
    for chunk in chunks:
        print(model.count_tokens(chunk).total_tokens)

    return chunks
#リクエストの処理
def send_request(chat_session,prompt):
    try:
        response = chat_session.send_message(prompt)
    except ResourceExhausted as e:
        print("Resource exhausted. Waiting for 1 minute before retrying...")
        for i in tqdm(range(60), desc="Waiting", unit="second"):
            time.sleep(1)
        response = chat_session.send_message(prompt)
    except Exception as e:
        print(f"An error occurred: {e}")
        raise
    return response
# 使用例
text_chunks = split_text_by_tokens(file_path)

In [ ]:
len(text_chunks)

In [ ]:
history = []
responses = []
for i, chunk in enumerate(text_chunks):
    chat_session = model.start_chat(history=history)
    response = send_request(chat_session,chunk)
    user_history = {"role": "user", "parts": chunk}
    model_history = {"role": "model", "parts": response.text}
    history.append(user_history)
    history.append(model_history)
    responses.append(response.text)
    print(f"Chunk {i+1} response:")

In [ ]:
#responsesを結合してファイルに書き込む
with open(output_path, 'w', encoding='utf-8') as file:
    for response in responses:
        file.write(response + '\n')
        